In [1]:
import cv2
import mediapipe as mp
import math
import numpy as np

In [2]:
# Initialize MediaPipe Pose
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [3]:
# Function to calculate the angle between three points
def calculate_angle(a, b, c):
    a = np.array(a)  # First point
    b = np.array(b)  # Mid point
    c = np.array(c)  # End point

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(
        a[1] - b[1], a[0] - b[0]
    )
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle


# Function to count right hand bicep curls
def count_bicep_curls(result):
    global bicep_counter, bicep_stage, left_bicep_stage, right_bicep_stage

    # Get the landmarks
    landmarks = result.pose_landmarks.landmark

    # Get the coordinates of the relevant landmarks
    right_shoulder = [
        landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
        landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y,
    ]
    right_elbow = [
        landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
        landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y,
    ]
    right_wrist = [
        landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
        landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y,
    ]

    left_shoulder = [
        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y,
    ]
    left_elbow = [
        landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
        landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y,
    ]
    left_wrist = [
        landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
        landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y,
    ]

    # Calculate the angle between the shoulder, elbow, and wrist
    right_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
    left_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)

    # Check if the arm is in the up or down position
    if right_angle > 160:
        right_bicep_stage = "down"
    elif right_angle < 30 and bicep_stage == "down":
        right_bicep_stage = "up"
        bicep_counter += 1

    if left_angle > 160:
        left_bicep_stage = "down"
    elif left_angle < 30 and bicep_stage == "down":
        left_bicep_stage = "up"
        bicep_counter += 1

    if left_bicep_stage == "up" or right_bicep_stage == "up":
        bicep_stage = "up"
    else:
        bicep_stage = "down"

    return bicep_counter, bicep_stage


# Function to count leg raises
def count_leg_raises(result):
    global leg_counter, left_leg_stage, right_leg_stage

    # Get the landmarks
    landmarks = result.pose_landmarks.landmark

    # Get the coordinates of the relevant landmarks
    hip_right = [
        landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
        landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y,
    ]
    knee_right = [
        landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
        landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y,
    ]
    ankle_right = [
        landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
        landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y,
    ]

    hip_left = [
        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y,
    ]
    knee_left = [
        landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
        landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y,
    ]
    ankle_left = [
        landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
        landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y,
    ]

    # Calculate the angle between the hip, knee, and ankle
    angle_right = calculate_angle(hip_right, knee_right, ankle_right)
    angle_left = calculate_angle(hip_left, knee_left, ankle_left)

    # Check if the leg is in the up or down position
    if angle_right > 170:
        left_leg_stage = "down"
    elif angle_right < 100 and left_leg_stage == "down":
        left_leg_stage = "up"
        leg_counter += 1

    if angle_left > 170:
        right_leg_stage = "down"
    elif angle_left < 100 and right_leg_stage == "down":
        right_leg_stage = "up"
        leg_counter += 1

    if left_leg_stage == "up" or right_leg_stage == "up":
        leg_stage = "up"
    else:
        leg_stage = "down"

    return leg_counter, leg_stage



In [9]:

# Initialize counters and stages
bicep_counter = 0
bicep_stage = None
leg_counter = 0
leg_stage = None

cap = cv2.VideoCapture(0) 

# Process frames
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert the frame to RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the frame with MediaPipe Pose
        result = pose.process(frame_rgb)

        # Draw pose landmarks on the frame
        if result.pose_landmarks:
            mp_drawing.draw_landmarks(
                frame, result.pose_landmarks, mp_pose.POSE_CONNECTIONS
            )

            # Count bicep curls and leg raises
            bicep_counter, bicep_stage = count_bicep_curls(result)
            leg_counter, leg_stage = count_leg_raises(result)

            # Display the counters and stages on the frame
            cv2.putText(
                frame,
                f"Bicep Counter: {bicep_counter}",
                (25, 50),
                cv2.FONT_HERSHEY_SIMPLEX,
                .4,
                (255, 255, 255),
                1,
                cv2.LINE_AA,
            )
            cv2.putText(
                frame,
                f"Bicep Stage: {bicep_stage}",
                (25, 75),
                cv2.FONT_HERSHEY_SIMPLEX,
                .4,
                (255, 255, 255),
                1,
                cv2.LINE_AA,
            )
            cv2.putText(
                frame,
                f"Leg Counter: {leg_counter}",
                (25, 100),
                cv2.FONT_HERSHEY_SIMPLEX,
                .4,
                (255, 255, 255),
                1,
                cv2.LINE_AA,
            )
            cv2.putText(
                frame,
                f"Leg Stage: {leg_stage}",
                (25, 125),
                cv2.FONT_HERSHEY_SIMPLEX,
                .4,
                (255, 255, 255),
                1,
                cv2.LINE_AA,
            )

        # Display the frame
        cv2.imshow("Pose Estimation", frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

cap.release()
cv2.destroyAllWindows()
